In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

# Chemical Energetics and Kinetics Virtual Notebook

Copyright **Peter Kraus and Paolo Raiteri**, January 2021


## First Virtual Lab - Statistics and Linear Regression

- [Access Workshop](virtual_workshop_01.ipynb): Linear Regression

## Second Virtual Lab - Kinetics

- [Exercise #1](kinetics_01.ipynb): Reaction of nitrogen oxide, nitrogen dioxide and oxygen
- [Exercise #2](kinetics_02.ipynb): Decomposition of benzene diazonium chloride
- [Exercise #3](kinetics_03.ipynb): Absorbance
- [Exercise #4](kinetics_04.ipynb): Decomposition reaction
- [Exercise #5](kinetics_05.ipynb): Gas phase decomposition
- [Exercise #6](kinetics_07.ipynb): Decompositio reaction
- [Exercise #7](kinetics_06.ipynb): Activation Energy

## Third Virtual Lab - Equilibrium

- [Exercise #1](kinetics_08.ipynb): Activation energy
- [Exercise #2](equilibrium_01.ipynb): Ideal chemical equilibrium
- [Exercise #3](equilibrium_02.ipynb): Non-ideal behaviour
- [Exercise #4](w4_03.ipynb): Enzyme kinetics


## Fourth Workshop - Self Consistent Problems

- [Exercise #1](w4_01.ipynb): Reaching equilibrium 
- [Exercise #2](w4_02.ipynb): Common ion equilibrium
- [Exercise #3](w4_03.ipynb): Phosphate ion equilibrium
